In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import DBSCAN
import sys
print(sys.executable)
import chess
import sqlite3
import chess.pgn
import os
import ast
import subprocess
import time
import csv
import traceback
from pathlib import Path
from tqdm import tqdm
from typing import Tuple, List, Dict, Optional,Callable,Any
import platform
from datetime import datetime
from huggingface_hub import notebook_login
from datasets import load_dataset
from prefect import flow, task
from collections import defaultdict
import seaborn as sns
from fuzzywuzzy import process
import hdbscan
from sklearn.manifold import TSNE
import umap.umap_ as umap
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans,DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture

db_file=r"C:\sqlite3\chess.db"
conn=sqlite3.connect(db_file)
df1=pd.read_sql_query("SELECT * FROM players",conn)
df2=pd.read_sql_query("SELECT * FROM player_piece_life",conn)
conn.close()
df2_temp = df2.copy()
df2_temp.rename(columns={"player": "user"}, inplace=True)
player_merged = df1.merge(df2_temp, on="user", how="left")
dfc=player_merged[player_merged['TotalGames']>100].copy()
dfc

c:\Users\Administrator\AppData\Local\Programs\Python\Python312\python.exe


,user,CenterControlScore,PieceActivityScore,KingSafetyScore,CastlingScore,KingTropismScore,KingDefendersScore,KingPawnShieldScore,KingZoneControlScore,KingDiagonalExposureScore,...,bishop_promo_7LifeRatio,bishop_promo_8LifeRatio,knight_promo_1LifeRatio,knight_promo_2LifeRatio,knight_promo_3LifeRatio,knight_promo_4LifeRatio,knight_promo_5LifeRatio,knight_promo_6LifeRatio,knight_promo_7LifeRatio,knight_promo_8LifeRatio
10,4empechement,20.798472396067886,194.08675547229439,-61.39023090984241,-14.78402609879107,-12.004720455579742,3.998456121762301,-6.018674604318203,-34.83847316082871,-9.986714553767536,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
11,AFRUZA Khamdamova,19.980144814965005,187.43250928135018,-62.598380640101944,-15.687360501452762,-11.541572491351232,3.9920080023441766,-3.880380874601807,-36.198732947141075,-10.5641031520965,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
17,Aadit Bhatia,18.949722544656208,179.93313300923768,-64.25990598217915,-14.957623817306374,-12.46291619937623,3.9382888251469295,-6.256986489149169,-35.260646895849455,-10.354357776354627,...,None,None,0.0,NaN,NaN,NaN,NaN,NaN,None,None
18,Aaditya Dhingra,19.75778663356812,186.0420885053482,-58.621692108406734,-14.993523775081961,-11.531950627869572,3.991304558505241,-5.895971274224948,-31.923962814203367,-10.118627841615467,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
21,Aakaash Meduri,19.224421016310867,183.42884121982763,-66.11922312432546,-15.097293801514592,-12.714093170189011,3.991397154889973,-7.024927197391557,-36.380611886077006,-10.544953751596001,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8806,Эдуардо Гербелли Нето,19.55056444908693,181.29926765246034,-59.57322353261568,-14.251761927932476,-11.658740243443402,3.9963650403227726,-6.894892390639647,-32.738635356478575,-10.374994590780966,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
8807,Эмир Шаршенбеков,19.802088160600384,180.49468697153765,-65.48740196115436,-15.759800427658622,-12.374676904935187,3.9906644738959423,-7.5830339415453425,-35.45868812404026,-10.573679055562021,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
8810,Яков Гинзбург,18.91048007399788,177.05044719361808,-57.17676036324323,-14.456618847750025,-11.312831307295832,3.996480827755714,-4.81888478126306,-32.28695870561078,-10.170877318691563,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
8811,Ян Дьомін,19.880803406283746,184.47318182362616,-61.429340310268365,-15.349555813948244,-11.737261132620924,3.997623991465466,-5.24869130929493,-34.791661388037674,-10.386153849094322,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [2]:
dfc.to_csv('player_features.csv')

In [4]:
player_features=pd.read_csv(r"C:\Users\Administrator\Desktop\playerstyles\Chess\player-cluster\player_features.csv")
df_elo=pd.read_csv(r"C:\Users\Administrator\Desktop\playerstyles\Chess\basic\newestELO\name_to_elos.csv")
player_features=player_features.merge(df_elo,left_on='user',right_on="Name", how="left")
player_features

,Unnamed: 0_x,user,CenterControlScore,PieceActivityScore,KingSafetyScore,CastlingScore,KingTropismScore,KingDefendersScore,KingPawnShieldScore,KingZoneControlScore,...,knight_promo_2LifeRatio,knight_promo_3LifeRatio,knight_promo_4LifeRatio,knight_promo_5LifeRatio,knight_promo_6LifeRatio,knight_promo_7LifeRatio,knight_promo_8LifeRatio,Unnamed: 0_y,Name,ELO
0,10,4empechement,20.798472,194.086755,-61.390231,-14.784026,-12.004720,3.998456,-6.018675,-34.838473,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,4empechement,2522.0
1,11,AFRUZA Khamdamova,19.980145,187.432509,-62.598381,-15.687361,-11.541572,3.992008,-3.880381,-36.198733,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,AFRUZA Khamdamova,2682.0
2,17,Aadit Bhatia,18.949723,179.933133,-64.259906,-14.957624,-12.462916,3.938289,-6.256986,-35.260647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,Aadit Bhatia,2528.0
3,18,Aaditya Dhingra,19.757787,186.042089,-58.621692,-14.993524,-11.531951,3.991305,-5.895971,-31.923963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,Aaditya Dhingra,2694.0
4,21,Aakaash Meduri,19.224421,183.428841,-66.119223,-15.097294,-12.714093,3.991397,-7.024927,-36.380612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Aakaash Meduri,2519.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,8806,Эдуардо Гербелли Нето,19.550564,181.299268,-59.573224,-14.251762,-11.658740,3.996365,-6.894892,-32.738635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9595,Эдуардо Гербелли Нето,2352.0
2691,8807,Эмир Шаршенбеков,19.802088,180.494687,-65.487402,-15.759800,-12.374677,3.990664,-7.583034,-35.458688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9597,Эмир Шаршенбеков,2718.0
2692,8810,Яков Гинзбург,18.910480,177.050447,-57.176760,-14.456619,-11.312831,3.996481,-4.818885,-32.286959,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9600,Яков Гинзбург,2562.0
2693,8811,Ян Дьомін,19.880803,184.473182,-61.429340,-15.349556,-11.737261,3.997624,-5.248691,-34.791661,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9601,Ян Дьомін,2855.0


In [7]:
player_features = player_features.drop(
    columns=['Node', 'Unnamed: 0','Unnamed: 0_x','Unnamed: 0_y' ],
    errors='ignore'
)
player_features.to_csv('player_features.csv')

In [ ]:
df_community=pd.read_csv(r"C:\Users\Administrator\Desktop\playerstyles\Chess\player-cluster\community_visualization_data.csv")
df_elo=pd.read_csv(r"C:\Users\Administrator\Desktop\playerstyles\Chess\basic\newestELO\name_to_elos.csv")
df_community_new=df_community.merge(df_elo,left_on='Node',right_on="Name", how="left")
df_community_new = df_community_new.drop(
    columns=['Node', 'X', 'Y', 'Unnamed: 0'],
    errors='ignore'
)
df_community_new
df_community_new.to_csv("df_community_new.csv", index=False)